# Seminar: Diphone Synthersis
At this seminar we will construct the simpliest possible synthesis - diphone model.
<img src="concat-scheme.png">
We will use part of the LJSpeech dataset.
Your task will be to design search and concatenation of the units.
Preprocessor stages are already performed for the test samples (and it'll be your home assignment to create a small g2p for CMU english phoneset).

## Alignment
The first and very import part in the data preparation is alignment: we need to determine the timings of phonemes our utterance consists of.
Even the concatenative syntheses are not used today in prod alignment is still an important phase for upsampling-based parametric acoustic models (e.g. fastspeech).

### Motreal Force Aligner
To process audio we will use MFA.

At the alignment stage we launch xent-trained TDNN ASR system with fixed text on the output and try to determine the most probable phonemes positions in the timeline.

In [1]:
%%writefile install_mfa.sh
#!/bin/bash

## a script to install Montreal Forced Aligner (MFA)

root_dir=${1:-/tmp/mfa}
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

# create py38 env
$root_dir/miniconda3/bin/conda create -n aligner -c conda-forge openblas libopenblas python=3.8 openfst pynini ngram baumwelch -y
source $root_dir/miniconda3/bin/activate aligner

# install mfa, download kaldi
pip install montreal-forced-aligner # install requirements
pip install git+https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner.git # install latest updates

mfa thirdparty download

echo -e "\n======== DONE =========="
echo -e "\nTo activate MFA, run: source $root_dir/miniconda3/bin/activate aligner"
echo -e "\nTo delete MFA, run: rm -rf $root_dir"
echo -e "\nSee: https://montreal-forced-aligner.readthedocs.io/en/latest/aligning.html to know how to use MFA"

Overwriting install_mfa.sh


In [2]:
# download and install mfa
INSTALL_DIR="/tmp/mfa" # path to install directory

!bash ./install_mfa.sh {INSTALL_DIR}

Miniconda3-latest-L 100%[===================>]  63.62M  17.0MB/s    in 3.9s    
PREFIX=/tmp/mfa/miniconda3
Unpacking payload ...
Solving environment: done

# All requested packages already installed.

installation finished.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /tmp/mfa/miniconda3/envs/aligner

  added / updated specs:
    - baumwelch
    - libopenblas
    - ngram
    - openblas
    - openfst
    - pynini
    - python=3.8


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-1_gnu
  atk-1.0            conda-forge/linux-64::atk-1.0-2.36.0-h3371d22_4
  baumwelch          conda-forge/linux-64::baumwelch-0.3.6-h4bd325d_0
  ca-certificates    conda-forg

  Using cached SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
  Using cached numba-0.53.1-cp38-cp38-manylinux2014_x86_64.whl (3.4 MB)
  Using cached scipy-1.7.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.4 MB)
  Using cached packaging-21.0-py3-none-any.whl (40 kB)
  Using cached pooch-1.4.0-py3-none-any.whl (51 kB)
  Using cached llvmlite-0.36.0-cp38-cp38-manylinux2010_x86_64.whl (25.3 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached threadpoolctl-2.2.0-py3-none-any.whl (12 kB)
  Using cached cffi-1.14.6-cp38-cp38-manylinux1_x86_64.whl (411 kB)
  Using cached pycparser-2.20-py2.py3-none-any.whl (112 kB)
  Using cached PyQt5_Qt5-5.15.2-py3-none-manylinux2014_x86_64.whl (59.9 MB)
  Using cached PyQt5_sip-12.9.0-cp38-cp38-manylinux1_x86_64.whl (332 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Using cached id

81.9MB [00:04, 18.6MB/s]                                                        

======== DONE ==========

To activate MFA, run: source /tmp/mfa/miniconda3/bin/activate aligner

To delete MFA, run: rm -rf /tmp/mfa

See: https://montreal-forced-aligner.readthedocs.io/en/latest/aligning.html to know how to use MFA


In [3]:
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; mfa align --help

usage: mfa align [-h] [--config_path CONFIG_PATH] [-s SPEAKER_CHARACTERS]
                 [-t TEMP_DIRECTORY] [-j NUM_JOBS] [-v] [-c] [-d]
                 corpus_directory dictionary_path acoustic_model_path
                 output_directory

positional arguments:
  corpus_directory      Full path to the directory to align
  dictionary_path       Full path to the pronunciation dictionary to use
  acoustic_model_path   Full path to the archive containing pre-trained model
                        or language ()
  output_directory      Full path to output directory, will be created if it
                        doesn't exist

optional arguments:
  -h, --help            show this help message and exit
  --config_path CONFIG_PATH
                        Path to config file to use for alignment
  -s SPEAKER_CHARACTERS, --speaker_characters SPEAKER_CHARACTERS
                        Number of characters of file names to use for
                        determining speaker, default is to use 

### LJSpeech data subset
Here we will download the dataset.
However we don't need the whole LJSpeech for diphone synthesis (and it will be processed for quite a while).
Here we will take about 1/10 of the dataset. That's more than enough for diphone TTS.

In [ ]:
!echo "download and unpack ljs dataset"
!mkdir -p ./ljs; cd ./ljs; wget -q --show-progress https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!cd ./ljs; tar xjf LJSpeech-1.1.tar.bz2

download and unpack ljs dataset
LJSpeech-1.1.tar.bz   2%[                    ]  60.76M  3.03MB/s    eta 14m 14s^C


In [5]:
# We need sox to convert audio to 16kHz (the format alignment works with)
!sudo apt install -q -y sox
!sudo apt install -q -y libopenblas-dev

[sudo] password for ishalyminov: 
[sudo] password for ishalyminov: 


In [6]:
!rm -rf ./wav
!mkdir ./wav
!cat wavs_need.txt | xargs -I F -P 30 sox --norm=-3 ./ljs/LJSpeech-1.1/wavs/F.wav -r 16k -c 1 ./wav/F.wav
!echo "Number of clips" $(ls ./wav/ | wc -l)

Number of clips 1273


It should be 1273 clips here

In [7]:
with open('wavs_need.txt') as ifile:
    wavs_need = {l.strip() for l in ifile}

In [8]:
# metadata to transcripts
lines = open('./ljs/LJSpeech-1.1/metadata.csv', 'r').readlines()
for line in lines:
    fn, _, transcript = line.strip().split('|')
    if fn in wavs_need:
        with open('./wav/{}.txt'.format(fn), 'w') as ofile:
            ofile.write(transcript)

!echo "Number of transcripts" $(ls ./wav/*.txt | wc -l)

Number of transcripts 1273


Let's download the artifacts for alignment.

For phoneme ASR we need acoustic model and lexicon (mapping word=>phonemes) made by some other g2p

In [22]:
!wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/main/acoustic/english.zip
!wget -q --show-progress http://www.openslr.org/resources/11/librispeech-lexicon.txt

english.zip         100%[===================>]  14.05M  18.5MB/s    in 0.8s    
librispeech-lexicon 100%[===================>]   5.37M  7.72MB/s    in 0.7s    


Finally, we come to the alignment.

It will take about 15-17 min for our subset to be aligned

In [9]:
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa align -t ./temp -c -j 4 ./wav librispeech-lexicon.txt ./english.zip ./ljs_aligned
!echo "See output files at ./ljs_aligned"

All required kaldi binaries were found!
Cleaning old directory!
./temp/wav/align.log
INFO - Setting up corpus information...
INFO - Number of speakers in corpus: 1, average number of utterances per speaker: 1273.0
INFO - Parsing dictionary without pronunciation probabilities without silence probabilities
INFO - Creating dictionary information...
INFO - Setting up training data...
Generating base features (mfcc)...
Calculating CMVN...
INFO - Done with setup!
INFO - Performing first-pass alignment...
INFO - Calculating fMLLR for speaker adaptation...
INFO - Performing second-pass alignment...
INFO - All done!
See output files at ./ljs_aligned


In [10]:
!ls ljs_aligned/ | wc -l

1273


In [11]:
import IPython.display
from IPython.core.display import display

def display_audio(data):
    display(IPython.display.Audio(data, rate=22050))

In [12]:
# to install textgrids
!pip install praat-textgrids

In [13]:
import numpy as np
from scipy.io import wavfile
import textgrids
import glob

Alignment outputs are textgrids - and xml-like structure with layers for phonemes and words (with timings)

In [14]:
alignment = {f.split("/")[-1].split(".")[0][4:]: textgrids.TextGrid(f) for f in glob.iglob('ljs_aligned/*')}

In [16]:
wavs = {f.split("/")[-1].split(".")[0]: wavfile.read(f)[1] for f in glob.iglob('./ljs/LJSpeech-1.1/wavs/*.wav')}

/home/ishalyminov/data/miniconda3/envs/shad_speech/lib/python3.7/site-packages/ipykernel_launcher.py:1: WavFileWarning: Reached EOF prematurely; finished at 71680 bytes, expected 410470 bytes from header.
  """Entry point for launching an IPython kernel.


In [17]:
allphones = {
    ph.text for grid in alignment.values() for ph in grid["phones"]
}
# let's exclude special symbols: silence, spoken noise, non-spoken noise
allphones = {ph for ph in allphones if ph and ph == ph.upper()}
assert len(allphones) == 69

Here your part begins:
You need to create `diphone index` - mapping structure that will allow you to find original utterance and position in it by diphone text id.

E.g.:
`index[(PH1, PH2)] -> (utt_id, phoneme_index)`

In [18]:
diphone_index = dict()
# !!!!!!!!!!!!!!!!!!!!!!#
for utt, grid in alignment.items():
    phones = [phone.text for phone in grid['phones']]
    for idx, (phone_i, phone_j) in enumerate(zip(phones[:-1], phones[1:])):
        diphone_index[phone_i, phone_j] = utt, idx
# !!!!!!!!!!!!!!!!!!!!!!#

In [19]:
# check yourself
for a, b in [('AH0', 'P'), ('P', 'AH0'), ('AH0', 'L')]:
    k, i = diphone_index[(a,b)]
    assert a == alignment[k]['phones'][i].text
    assert b == alignment[k]['phones'][i+1].text

In concat TTS you sometimes don't have all the diphones presented
If it's not very frequent ones it's not a trouble
But we need to provide some mechanism to replace missing units

In [20]:
with open("fallback_rules.txt") as ifile:
    lines = [l.strip().split() for l in ifile]
    fallback_rules = {l[0]: l[1:] for l in lines}

In the dict `fallback_rules` lie possible replacement for all the phones
(different replacements in order of similarity).

E.g. `a stressed` -> `a unstressed`  | `o stressed` | `o unstressed`

Here is also some work for you:
You need to create diphone fallbacks from the phoneme ones:

`diphone_fallbacks[(Ph1, Ph2)] -> (some_other_pair_of_phones_presented_in_dataset)`

and also, if `diphone_fallbacks[(a, b)] = c, d` then:
* c = a or
* c $\in$ fallback_rules[a] and/or
* d = b or
* d $\in$ fallback_rules[d]


In [21]:
diphone_fallbacks = dict()
# !!!!!!!!!!!!!!!!!!!!!!#
diphone_fallbacks[('Z', 'Z')] = ('ZH', 'ZH')
diphone_fallbacks[('Z', 'AY1')] = ('Z', 'EY1')
diphone_fallbacks[('Z', 'EY0')] = ('ZH', 'EH1')
# !!!!!!!!!!!!!!!!!!!!!!#

In [22]:
# check yourself
for a, b in [('Z', 'Z'), ('Z', 'AY1'), ('Z', 'EY0')]:
    assert (a, b) in diphone_fallbacks
    r1, r2 = diphone_fallbacks[(a, b)]
    assert r1 in fallback_rules[a] or r1 == a
    assert r2 in fallback_rules[b] or r2 == b
    assert r1 != a or r2 != b

In [23]:
# some helping constants
SAMPLE_RATE = 22050
WAV_TYPE = np.int16

Little DSP related to concatenative synthesis:

to prevent disturbing "clicking" sound (difference in volume) when concatenating fragments from different utterances we need to perform `cross-fade` - smoothing at concatenation point

If we concatenate $wav_1$ and $wav_2$ at some points $M_1$ and $M_2$ corrispondively we perform crossfade with overlap of $2 V$:

$$\forall i \in [-V; V]:~output[M_1+i] = (1-\alpha) \cdot wav_1[M_1+i] + \alpha \cdot wav_2[M_2+i]$$
Where $$\alpha = \frac{i+V}{2 V}$$

And for $i < -V:~ output[M_1+i] = wav_1[M_1+i]$

for $i > V:~output[M_1+i] = wav_2[M_2+i]$


But it is not ok if the overlapping comes outside the concatenation phoneme.

So, if junction phoneme starts and ends at positions $B_1$ and $E_1$ (the first wav) and $B_2$ and $E_2$ (the second one)
the extact formula for overlapping zone will be:
$$\forall i \in [-L; R]:~output[M_1+i] = (1-\alpha) \cdot wav_1[M_1+i] + \alpha \cdot wav_2[M_2+i]$$
Where:
$$\alpha = \frac{i+L}{L+R},~L = min(M_1-B_1, M_2 - B_2, V), ~R = min(E_1-M_1, E_2-M_2, V)$$
    

In [267]:
def crossfade(lcenter, ldata, rcenter, rdata, halfoverlap):
    """
    ldata, rdata - 1d numpy array only with junction phoneme (so, B1 = 0, E1 = ldata.shape[0])
    lcenter = M1
    rcenter = M2
    
    it is better to return the concatenated version of the junction phoneme (as numpy data)
    """
    output = np.zeros(lcenter + rcenter)
    # !!!!!!!!!!!!!!!!!!!!!!#
    lmask = np.zeros(len(ldata))
    lmask[:lcenter - halfoverlap] = 1.0
    lmask[lcenter - halfoverlap: lcenter + halfoverlap] = np.linspace(1, 0, 2 * halfoverlap)

    rmask = np.zeros(len(rdata))
    rmask[-rcenter + halfoverlap:] = 1.0    
    rmask[-rcenter - halfoverlap: -rcenter + halfoverlap] = np.linspace(0, 1, 2 * halfoverlap)
    
    output[:lcenter + halfoverlap] += (ldata * lmask)[:lcenter + halfoverlap]
    output[-rcenter - halfoverlap:] += (rdata * rmask)[-rcenter - halfoverlap:]
    return output
    # !!!!!!!!!!!!!!!!!!!!!!#

In [81]:
def get_data(k, i):
    phoneme = alignment[k]['phones'][i]
    left = phoneme.xmin
    right = phoneme.xmax
    center = (left+right) * .5
    
    left = int(left * SAMPLE_RATE)
    center = int(center * SAMPLE_RATE)
    right = int(right * SAMPLE_RATE)
    return center - left, wavs[k][left:right]

In [242]:
# check yourself
cf = crossfade(*get_data('LJ050-0241', 3), *get_data('LJ038-0067', 56), 300)
assert np.abs(cf.shape[0] - 1764) < 10
assert np.abs(cf.mean() - 11) < 0.1

In [176]:
len(get_data('LJ050-0241', 3)[1]), get_data('LJ050-0241', 3)[0]

(1324, 662)

In [175]:
len(get_data('LJ038-0067', 56)[1]), get_data('LJ038-0067', 56)[0]

(2205, 1103)

In [201]:
display_audio(get_data('LJ050-0241', 3)[1])

In [195]:
display_audio(get_data('LJ038-0067', 56)[1])

In [254]:
display_audio(cf)

In [275]:
HALF_OVERLAP_CROSSFADE = 300

def synthesize(phonemes):
    diphones = []
    for ph1, ph2 in zip(phonemes[:-1], phonemes[1:]):
        diphone = (ph1, ph2)
        if diphone in diphone_index:
            k, i = diphone_index[diphone]
        else:
            k, i = diphone_index[diphone_fallbacks[diphone]]
            
        diphones.append((get_data(k, i), get_data(k, i+1)))
    output = []
    
    # Here you need to construct the result utterance with crossfades
    # NB: border (the first and the last phonemes does not require any crossfade and could be just copied)
    # !!!!!!!!!!!!!!!!!!!!!!#
    ctx_lcenter = 0
    for idx, diphone in enumerate(diphones):
        (lcenter, ldata), (rcenter, rdata) = diphone
        if idx == 0:
            output += list(ldata)
            ctx_lcenter = lcenter
            continue
        elif idx == len(diphones) - 1:
            output += list(rdata)
            continue
        output = list(crossfade(ctx_lcenter, output, lcenter, ldata, HALF_OVERLAP_CROSSFADE))
        ctx_lcenter = len(output) - HALF_OVERLAP_CROSSFADE
        output = list(crossfade(ctx_lcenter, output, rcenter, rdata, HALF_OVERLAP_CROSSFADE))
        ctx_lcenter = len(output) - 2 * HALF_OVERLAP_CROSSFADE
    return np.array(output, dtype=WAV_TYPE)
    # !!!!!!!!!!!!!!!!!!!!!!#
    # need to return wav as 1d numpy array of type WAV_TYPE

Check youself:

If everything was correct, you should hear 'hello world'

In [276]:
display_audio(synthesize(['HH', 'AH0', 'L', 'OW1', 'W', 'ER1', 'L', 'D']))

In [128]:
# load additional test texts
with open("test_phones.txt") as ifile:
    test_phones = []
    for l in ifile:
        test_phones.append(l.strip().split())

Here should a little part of the GLADOS song 

In [277]:
output = []
pause = np.zeros([int(0.1 * SAMPLE_RATE)], dtype=WAV_TYPE)
for test in test_phones:
    output.append(synthesize(test))
    output.append(pause)
    
display_audio(np.concatenate(output[:-1]))